# Shadows with xthreejs

This example is meant to demonstrate how to set up shadows with pythreejs. It is mainly based on the example code you can find in the three.js documentation, but is adapted to highlight some of the nuances of using it from xthreejs.

## Setup

First, we set up an example scene for exploring shadow behavior.

In [1]:
#include <xthreejs/xthreejs.hpp>

In [2]:
std::size_t view_width = 800;
std::size_t view_height = 600;

Create some example geometry in nice coder colors:

In [3]:
auto sphere = xthree::mesh_generator()
    .geometry(xthree::sphere_buffer_geometry_generator()
              .radius(1)
              .widthSegments(32)
              .heightSegments(16)
              .finalize()
              )
    .material(xthree::mesh_standard_material_generator()
              .color("red")
              .finalize()
              )
    .finalize();

In [4]:
auto cube = xthree::mesh_generator()
    .geometry(xthree::box_buffer_geometry_generator()
              .width(1)
              .height(1)
              .depth(1)
              .finalize()
              )
    .material(xthree::mesh_physical_material_generator()
              .color("green")
              .finalize()
              )
    .position({2, 0, 4})
    .finalize();

In [5]:
auto plane = xthree::mesh_generator()
    .geometry(xthree::plane_buffer_geometry_generator()
              .width(10)
              .height(10)
              .finalize()
              )
    .material(xthree::mesh_standard_material_generator()
              .color("gray")
              .finalize()
              )
    .position({0, -2, 0})
    .finalize();

In [6]:
plane.rotation = xthree::euler(-3.14/2, 0, 0, "XYZ");

Create camera and lighting:

In [7]:
auto camera = xthree::perspective_camera_generator()
    .position({10, 6, 10})
    .aspect(view_width/view_height)
    .finalize();
auto key_light = xthree::spot_light_generator()
    .position({0, 10, 10})
    .angle(0.3)
    .penumbra(0.1)
    .finalize();
auto ambient_light = xthree::ambient_light();

In [8]:
auto scene = xthree::scene_generator()
    .children({sphere, cube, plane, camera, key_light, ambient_light})
    .finalize();
auto controller = xthree::orbit_controls_generator()
    .controlling(camera)
    .finalize();
auto renderer = xthree::renderer_generator()
    .camera(camera)
    .scene(scene)
    .controls({controller})
    ._width(view_width)
    ._height(view_height)
    //.antialias(true)
    .finalize();

In [9]:
renderer.display()

A Jupyter widget

## Configuring shadows

Now we can start playing around with the shadows in such a way that the results of the different options are immediately shown in the rendered scene.

First, set the spot light to track the cube position:

In [10]:
key_light.target = cube;

Turn on shadows in the renderer:

In [11]:
renderer.shadowMap.enabled = true;
renderer.shadowMap.type = "PCFSoftShadowMap"  // default PCFShadowMap

input_line_17:2:21: error: no member named 'enabled' in 'xthree::xrender_widget<xw::xmaterialize<xthree::xrenderer> >::shadowMap_property'
 renderer.shadowMap.enabled = true;
 ~~~~~~~~~~~~~~~~~~ ^
input_line_17:3:20: error: no member named 'type' in 'xthree::xrender_widget<xw::xmaterialize<xthree::xrenderer> >::shadowMap_property'
renderer.shadowMap.type = "PCFSoftShadowMap"  // default PCFShadowMap
~~~~~~~~~~~~~~~~~~ ^


Interpreter Error: 

Even with shadow maps enabled, there are still no shadows. This is because three.js only includes those lights and objects that has been explicitly marked for shadows in its calculations. Let's turn on some of these:

In [12]:
// Enable shadows for the light
key_light.castShadow = true;

// Enable casting/receiving shadows for some objects:
sphere.castShadow = true;
cube.castShadow = true;
plane.receiveShadow = true;

Let's move the cube closer to the sphere:

In [12]:
cube.position = (0, 1, 2)

Note that the light changed to track the position of the cube. It is also clear that the shadow from the cube is not being taken into account on the sphere. As before, we can turn this on with `receiveShadow` on the sphere, but we also need to mark the sphere material for an update. This is needed for any shadow changes *after the first frame with shadows* is rendered.

In [13]:
# Also enable sphere to receive shadow:
sphere.receiveShadow = True
sphere.material.needsUpdate = True

Finally, let's zoom in on the details of the shadows:

In [14]:
camera.position = [2.92, 1.75, 2.92]
camera.quaternion = [-0.18, 0.38, 0.076, 0.90]

Here, we can see that there is some pixelation of the shadows (although it is smoothed by using `PCFSoftShadowMap`). This can be fixed by increasing the resolution of the shadow map:

In [15]:
key_light.shadow.mapSize = (2048, 2048)